In [1]:
import pandas as pd
import folium

In [3]:
all_clean = pd.read_csv('./input/all_clean.csv')

In [10]:
city_count = all_clean.groupby(['city']).count()
city_sorted = city_count.sort_values(['tech_startups'], ascending=[False])

In [38]:
city_sorted['tech_startups']

city
New York               52
San Francisco          52
Seattle                10
Boston                  6
Chicago                 5
Paris                   5
Toronto                 5
Mountain View           4
Palo Alto               4
Los Angeles             4
London                  3
Boulder                 3
Bellevue                3
Montreal                3
Reston                  2
Redwood City            2
Sydney                  2
Menlo Park              2
Amsterdam               2
Copenhagen              2
Berlin                  2
Corte Madera            2
Colorado Springs        1
Birmingham              1
Sunnyvale               1
Stockholm               1
Singapore               1
Budapest                1
Santa Monica            1
Santa Clara             1
San Mateo               1
San Jose                1
Cambridge               1
Lancaster               1
Honolulu                1
Pittsburgh              1
Emeryville              1
Olpe                    1
Glen Co

In [54]:
choose_city = str(input("Insert a city from the list above: "))

Insert a city from the list above: Honolulu


In [55]:
city_df = all_clean.loc[all_clean.city == choose_city]

In [56]:
lati = city_df['latitude'].iloc[0]
long = city_df['longitude'].iloc[0]
map_city=folium.Map(location=[float(lati),float(long)], zoom_start=12)

for index, row in city_df.iterrows():
    folium.Marker((row['latitude'],row['longitude']),
                    radius=2,
                    icon=folium.Icon(icon='home',color='red'),
                    tooltip='Cool Tech Startup: '+str(row['tech_startups'])
                   ).add_to(map_city)
    
for index, row in city_df.iterrows():
    folium.Marker((row['starbucks_lat'],row['starbucks_lon']),
                    radius=2,
                    icon=folium.Icon(icon='leaf',color='green'),
                    tooltip=str(row['starbucks'])
                   ).add_to(map_city)
    
for index, row in city_df.iterrows():
    folium.Marker((row['air_lat'],row['air_lon']),
                    radius=2,
                    icon=folium.Icon(icon='plane',color='lightgray'),
                    tooltip=str(row['airport'])
                   ).add_to(map_city)
    
    
for index, row in city_df.iterrows():
    folium.Marker((row['daycare_lat'],row['daycare_lon']),
                    radius=2,
                    icon=folium.Icon(icon='heart',color='blue'),
                    tooltip='Kindergarten: '+str(row['daycare'])
                   ).add_to(map_city)
    
    
for index, row in city_df.iterrows():
    folium.Marker((row['nightclub_lat'],row['nightclub_lon']),
                    radius=2,
                    icon=folium.Icon(icon='glass',color='darkpurple'),
                    tooltip='Nightclub: '+str(row['nightclub'])
                   ).add_to(map_city)
    
for index, row in city_df.iterrows():
    folium.Marker((row['vegan_restaurant_lat'],row['vegan_restaurant_lon']),
                    radius=2,
                    icon=folium.Icon(icon='cutlery',color='darkgreen'),
                    tooltip='Vegan: '+str(row['vegan_restaurant'])
                   ).add_to(map_city)

In [57]:
map_city